# Sembla Agent Demo

In [1]:
import os

from sembla.schemas.system import SystemState, Message
from sembla.system import init_sequential_agent_system
from sembla.conversation_history import update_conversation_history
from sembla.llm.openai.chat_completion import generate_chat_completion


%load_ext autoreload
%autoreload 2

# Set the OpenAI API key here or in your environment
OPENAI_API_KEY = None

if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


**Note:** User query should probably be a dict so that we can support things like multimodal systems.

In [2]:
user_query = "What is the capital of Ireland?"

initial_state = SystemState(user_query=user_query)
print("Initial State:")
print(initial_state.to_json())


Initial State:
{
    "task": {
        "name": null,
        "description": null,
        "status": "UNDEFINED",
        "max_cycles": null,
        "current_cycle": 0
    },
    "model": {
        "name": "gpt-3.5-turbo",
        "temperature": 0.2,
        "n": 1,
        "max_tokens": 2000,
        "frequency_penalty": 0,
        "presence_penalty": 0
    },
    "memory": {
        "max_history_message_count": 100,
        "max_history_token_count": 1000,
        "conversation_history": [],
        "conversation_buffer": [],
        "message_count": 0,
        "token_count": 0
    },
    "user_query": "What is the capital of Ireland?",
    "available_actions": [],
    "prompt_processor_outputs": [],
    "processed_prompt": null,
    "agent_response": null,
    "response_processor_outputs": [],
    "processed_response": null
}


In [3]:
def process_user_query(system_state: SystemState) -> SystemState:
    """Process the user query into a message and add it to the conversation buffer."""
    prompt = system_state.user_query
    assert prompt is not None
    message = Message(role="user", content=prompt)
    new_memory = system_state.memory.copy(update={"conversation_buffer": [message]})
    new_state = system_state.copy(update={
        "memory": new_memory,
        "processed_prompt": prompt}
    )
    return new_state

def process_agent_response(system_state: SystemState) -> SystemState:
    """Process the agent response into a message and add it to the conversation buffer."""
    response = system_state.agent_response
    assert response is not None
    message = Message(role="assistant", content=response)
    new_memory = system_state.memory.copy(update={"conversation_buffer": [message]})
    new_state = system_state.copy(update={
        "memory": new_memory,
        "processed_response": response}
    )
    return new_state


In [4]:
agent_system = init_sequential_agent_system([
    process_user_query,
    update_conversation_history,
    generate_chat_completion,
    process_agent_response,
    update_conversation_history,
])

In [5]:
final_state = agent_system(initial_state)
agent_response = final_state.agent_response
print("User Query:")
print(user_query)
print()
print("Agent Response:")
print(agent_response)
print()
print("Final State:")
print(final_state.to_json())

User Query:
What is the capital of Ireland?

Agent Response:
The capital of Ireland is Dublin.

Final State:
{
    "task": {
        "name": null,
        "description": null,
        "status": "UNDEFINED",
        "max_cycles": null,
        "current_cycle": 1
    },
    "model": {
        "name": "gpt-3.5-turbo",
        "temperature": 0.2,
        "n": 1,
        "max_tokens": 2000,
        "frequency_penalty": 0,
        "presence_penalty": 0
    },
    "memory": {
        "max_history_message_count": 100,
        "max_history_token_count": 1000,
        "conversation_history": [
            {
                "role": "user",
                "content": "What is the capital of Ireland?",
                "name": null
            },
            {
                "role": "assistant",
                "content": "The capital of Ireland is Dublin.",
                "name": null
            }
        ],
        "conversation_buffer": [],
        "message_count": 2,
        "token_count": 25